In [20]:
# Import of the packages
import numpy as np
import scipy.optimize as optimize
import matplotlib.pyplot as plt
import math
import scipy.stats as sts

In [21]:
class ParameterValueError(Exception):
    """ Custom exception raised when a parameter has an invalid value."""
    pass

In [152]:
# RVM class
class RVM:
    """ Relevance Vector Machine (RVM)
    
    Implementation of RVM for both regression and classification.
    
    Attributes:
        kerType: A string of the type of the desired kernel.
        rvmType: A string denoting the RVM type to be used. The string "EM" denotes
            an EM-like algorithm will be used to estimate the hyperparameters while
            the string "DD" denotes the direct differentiation approach.
        p: Integer value denoting the degree of the polynomial kernel.
        sigma: Float value denoting the smoothing factor of the Gaussian kernel.
        kappa: Float value denoting the scaling parameter of the sigmoid kernel.
        delta: Float value denoting the translation parameter of the sigmoid kernel.
        bTrained: boolean value which becomes true once the RVM has been trained.
    """

    EPSILON = 1e-10
   
    def __init__(self, kerType = 'poly', rvmType = "EM", p = 1, sigma = 1, 
                 kappa = 1, delta = 1):
        """ Initializes the RVM class (constructor).
        
            Raises:
                ParameterValueError: An error occured because a parameter had an
                    invalid value.  
        """
        # Check if the kernel type chosen is valid
        kerTypes = ['linear', 'poly', 'radial', 'sigmoid']
        if kerType not in kerTypes:
            raise ParameterValueError("ParameterValueError: The string " + kerType + \
                                       " does not denote a valid kernel type")
        # Check if the string denoting the rvmType has a valid value
        if rvmType != 'EM' and rvmType != 'DD':
            raise ParameterValueError('ParameterValueError: ' + rvmType, \
                                       " is not a valid RVM type value. Enter 'EM' or 'DD' as a value. ")
       
        self.kerType = kerType
        self.rvmType = rvmType
        self.p = p
        self.sigma = sigma
        self.kappa = kappa
        self.delta = delta
        self.bTrained = False

    def kernel(self, x, y):
        """ Kernel computation.
        
        It computes the kernel value based on the dot product
        between two vectors.
        
        Args:
            x: Input vector.
            y: Other input vector.
            
        Returns:
            The computed kernel value.
        """  
        if self.kerType == "linear":
            k = np.dot(x,y) + 1
        elif self.kerType == "poly":
            k = (np.dot(x,y) + 1) ** self.p
        elif self.kerType == "radial":
            k = math.exp(-(np.dot(x-y,x-y))/(2*self.sigma))
        elif self.kerType == "sigmoid":
            k = math.atanh(self.kappa * np.dot(x,y) - self.delta)

        return k
    
    def getKernelMatrix(self, X):
        """ Evaluates the kernel matrix K given a set of input samples.

            Args:
                X_tr: A NxM matrix with a M dimensional training input sample
                    in each row.
                
            Returns:
                An NxN Kernel matrix where N is the number of input samples.
        """
        N = X.shape
        K = np.zeros((N_tr, N_tr))
        for i in range(N_tr):
            for j in range(N_tr):
                K[i,j] = self.kernel(X[i], X[j])
        return K

    def train(self, X_tr, Y_tr):
        """ RVM training method
        
        Applies an EM-like algorithm or direct differentiation to estimate the
        optimal hyperparameters (alpha and sigma) needed to make predictions
        using the marginal likelihood.
        
        
        Args:
            X_tr: A matrix with a training input sample in each row.
            Y_tr: A vector with the output values of each input sample
                in X_tr.
        
        Returns:
            None
        """
        # Get number of training data samples
        N = X_tr.shape[0]
        # Initialize the alpha values (weight precision values) and the A matrix
        alpha_values = np.ones(N + 1)
        A = np.diag(alpha_values)
        # Initialize the sigma squared value and the B matrix
        sigma_squared = 1
        B = np.identity(N) * (sigma_squared ** -2)
        # Calculate kernel matrix K and append a column with ones in the front (for the bias term??)
        K = self.getKernelMatrix(X_tr)
        K = np.hstack((np.ones(N).reshape((N, 1)), K))

In [153]:
N_tr = 50
N_ts = 1000

def trans(x):
    N = np.shape(x)[0]
    y = np.sin(x) + 3 + np.random.normal(loc=0, scale=0.1, size=N)
    return y 

x_tr = np.linspace(0,7.5,num=N_tr)
y_tr = trans(x_tr)
x_ts = np.linspace(0,7.5,num=N_ts)
y_ts = trans(x_ts)

In [154]:
try:
    s = RVM()
    s.train(x_tr, y_tr)
except Exception as e:
    print(e)
    